In [22]:
!pip3 install scrapbook
!pip3 install recommenders

In [23]:
%load_ext autoreload
%autoreload 2

import logging
import numpy as np
import pandas as pd
import scrapbook as sb
from sklearn.preprocessing import minmax_scale

from recommenders.utils.python_utils import binarize
from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_stratified_split
from sklearn.model_selection import train_test_split
from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)
from recommenders.models.sar import SAR
import sys

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
System version: 3.9.16 (main, Jan 11 2023, 10:02:19) 
[Clang 14.0.6 ]
Pandas version: 1.5.3


In [24]:
clean_df = pd.read_csv('../Reco/Data/workspaces_clean.csv', index_col=0)
clean_df

,Workspace_Id,Name,Rating,Review_count,Price_range,Category,Address,Latitude,Longitude,Next_status
0,1,Eugenio Trias Municipal Public Library,3.8,800,0,Public library,"P.º de Fernán Núñez, 24",40.416705,-3.679161,Opens 8:30 AM Mon
1,2,Iván de Vargas Library,4.3,313,0,Public library,"C. de San Justo, 5",40.413991,-3.709750,Opens 8:30 AM Mon
2,3,Biblioteca Mario Vargas Llosa,3.8,178,0,Public library,"C. de Barceló, 4",40.426713,-3.699394,Opens 8:30 AM Mon
3,4,Pedro Salinas Library,4.0,337,0,Public library,"Gta. de la Prta de Toledo, 1",40.407074,-3.710894,Opens 9 AM Mon
4,5,Acuna Public Library,2.9,118,0,Public library,"C. de Quintana, 9",40.427932,-3.716937,Opens 9 AM Mon
...,...,...,...,...,...,...,...,...,...,...
113,261,Harina,3.9,434,2,Coffee shop,"C. de Velázquez, 61",40.429262,-3.684050,Closes 9 PM
114,262,The Coffee Corner,4.3,314,1,Coffee shop,"Av. de Valladolid, 41",40.428630,-3.729667,Closes 9 PM
115,263,The Bear and the Madroño,4.4,590,1,Espresso bar,"C. del Doce de Octubre, 16",40.415687,-3.675956,Closes 10:30 PM
116,264,Cafés Pozo,4.6,52,0,Coffee store,"C. de Miguel Arredondo, 4",40.394994,-3.695993,Closes 2 PMReopens 5 PM


In [25]:
# top k items to recommend
TOP_K = 10

Functions to generate a user ratings dataframe.

In [26]:
# function to get a random workspace for given seed as parameter
def generate_workspace(clean_df, random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    num_workspaces = len(clean_df)
    random_index = random.randint(0, num_workspaces - 1)
    random_row = clean_df.iloc[random_index,[0]]
    return random_row

In [27]:
def generate_rating(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    return round(np.random.uniform(1.0, 5.0), 1)

In [28]:
import datetime
from datetime import timedelta
import random

# function to generate a random date for transaction with a given seed as parameter
def generate_date(random_date_seed):
    
    # set random seed
    random.seed(random_date_seed) 
    
    # set start date to January 1, 2020
    start_date = datetime.datetime(2020, 1, 1) 
    
     # set end date tp current date
    end_date = datetime.datetime.now()
    
    # generate random date between start date and current date
    random_date = start_date + (end_date - start_date) * random.random()
    
    return random_date

In [29]:
import datetime
from datetime import timedelta
import random

# function to generate a random date for transaction with a given seed as parameter
def generate_timestamp(random_date_seed):
    
    # set random seed
    random.seed(random_date_seed) 
    
    # set start date to January 1, 2020
    start_date = datetime.datetime(2020, 1, 1) 
    
     # set end date tp current date
    end_date = datetime.datetime.now()
    
    # generate random date between start date and current date
    random_date = start_date + (end_date - start_date) * random.random()
    
    return round(datetime.datetime.timestamp(random_date))

In [30]:
# Get 100 row indices labels following pattern:
# User_1, User_2, User_3 ... User_100
user_row_indices = []
for i in range(1, 101):
    user_row_indices.append(i)


# set columns for user transactions data frame
cols = ["User_Id", "Workspace_Id", "Rating", "Timestamp", "Event"]

# create user transactions dataframe given columns
transactions_df = pd.DataFrame(columns = cols)

# two seeds are needed to generate random values: transaction_seed and workspace_date_rating_rating_seed
# transaction_seed to generate a random number of transactions for each user
# workspace_date_rating_rating_seed to genrate a workspace and date for each new transaction
transaction_seed, workspace_date_rating_rating_seed = 2023, 2023

for user in user_row_indices:
    
    # initialise transaction_seed
    random.seed(transaction_seed)
    np.random.seed(transaction_seed)
    
    # get number of transactions for each user with seed
    #num_trans = random.randint(1, 20)
    
    # generate n transactions for current user, n = num_trans
    for i in range(1, 101):
        
        # generate a random date with workspace_date_rating seed
        # result is a tuple ("workspace name", "category")
        workspace = generate_workspace(clean_df, workspace_date_rating_rating_seed)
        
        # generate a random rating with workspace_date_rating seed
        rating = generate_rating(workspace_date_rating_rating_seed)
        
        # generate a random workspace with workspace_date_rating seed
        timestamp = generate_timestamp(workspace_date_rating_rating_seed)
        
        # create transaction row with generated data
        row = pd.DataFrame({
            "User_Id": [user],
            "Workspace_Id": (workspace[0]),
            "Rating": [rating],
            "Timestamp": [timestamp],
            "Event": ["Click"]
        })
        
        # concatenate new transaction row to transaction_df
        transactions_df = pd.concat([transactions_df, row])
        
        # increment workspace_date_rating_rating_seed so that each new transaction
        # generates a different workspace and date
        # that stays the same each time the program is repeated
        workspace_date_rating_rating_seed += 1
        
    # increment transaction_seed so that each new user generates a new number of transaction
    # that stays the same each time the program is repeated
    transaction_seed += 1
    
transactions_df

,User_Id,Workspace_Id,Rating,Timestamp,Event
0,1,197,2.3,1616067681,Click
0,1,241,3.4,1624767823,Click
0,1,43,1.5,1633520757,Click
0,1,61,1.9,1589723636,Click
0,1,56,4.2,1588646591,Click
...,...,...,...,...,...
0,100,169,1.5,1639089694,Click
0,100,210,4.1,1618691031,Click
0,100,114,4.6,1657824780,Click
0,100,116,1.3,1600430978,Click


In [31]:
transactions_df.dtypes

User_Id          object
Workspace_Id     object
Rating          float64
Timestamp        object
Event            object
dtype: object

In [32]:
transactions_df["User_Id"] = transactions_df["User_Id"].astype("int64")
transactions_df["Workspace_Id"] = transactions_df["Workspace_Id"].astype("int64")
transactions_df.dtypes

User_Id           int64
Workspace_Id      int64
Rating          float64
Timestamp        object
Event            object
dtype: object

In [33]:
train, test = python_stratified_split(transactions_df, ratio=0.75, col_user='User_Id', col_item='Workspace_Id', seed=42)
print("""
Train:
Total Ratings: {train_total}
Unique Users: {train_users}
Unique Items: {train_items}

Test:
Total Ratings: {test_total}
Unique Users: {test_users}
Unique Items: {test_items}
""".format(
    train_total=len(train),
    train_users=len(train['User_Id'].unique()),
    train_items=len(train['Workspace_Id'].unique()),
    test_total=len(test),
    test_users=len(test['User_Id'].unique()),
    test_items=len(test['Workspace_Id'].unique()),
))


Train:
Total Ratings: 7500
Unique Users: 100
Unique Items: 264

Test:
Total Ratings: 2500
Unique Users: 100
Unique Items: 264



In [34]:
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SAR(
    col_user="User_Id",
    col_item="Workspace_Id",
    col_rating="Rating",
    col_timestamp="Timestamp",
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)

In [35]:
with Timer() as train_time:
    model.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

2023-03-01 15:54:53,493 INFO     Collecting user affinity matrix
2023-03-01 15:54:53,494 INFO     Calculating time-decayed affinities
2023-03-01 15:54:53,500 INFO     Creating index columns
2023-03-01 15:54:53,504 INFO     Calculating normalization factors
2023-03-01 15:54:53,509 INFO     Building user affinity sparse matrix
2023-03-01 15:54:53,510 INFO     Calculating item co-occurrence
2023-03-01 15:54:53,517 INFO     Calculating item similarity
2023-03-01 15:54:53,517 INFO     Using jaccard based similarity
2023-03-01 15:54:53,519 INFO     Done training


Took 0.02903770800000416 seconds for training.


In [36]:
with Timer() as test_time:
    top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

print("Took {} seconds for prediction.".format(test_time.interval))

2023-03-01 15:54:53,553 INFO     Calculating recommendation scores
2023-03-01 15:54:53,555 INFO     Removing seen items


Took 0.006090125000014268 seconds for prediction.


In [37]:
top_k.head()

,User_Id,Workspace_Id,prediction
0,1,13,0.328121
1,1,123,0.327976
2,1,132,0.315800
3,1,87,0.315228
4,1,72,0.312476


## Evaluate SAR model

In [38]:
eval_map = map_at_k(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating', k=TOP_K)

In [39]:
eval_ndcg = ndcg_at_k(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating', k=TOP_K)

In [40]:
eval_precision = precision_at_k(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating', k=TOP_K)

In [41]:
eval_recall = recall_at_k(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating', k=TOP_K)

In [42]:
eval_rmse = rmse(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [43]:
eval_mae = mae(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [44]:
eval_rsquared = rsquared(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [45]:
eval_exp_var = exp_var(test, top_k, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [46]:
positivity_threshold = 2
test_bin = test.copy()
test_bin['Rating'] = binarize(test_bin['Rating'], positivity_threshold)

top_k_prob = top_k.copy()
top_k_prob['prediction'] = minmax_scale(
    top_k_prob['prediction'].astype(float)
)

eval_logloss = logloss(test_bin, top_k_prob, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [47]:
positivity_threshold = 2
test_bin = test.copy()
test_bin['Rating'] = binarize(test_bin['Rating'], positivity_threshold)

top_k_prob = top_k.copy()
top_k_prob['prediction'] = minmax_scale(
    top_k_prob['prediction'].astype(float)
)

eval_logloss = logloss(test_bin, top_k_prob, col_user='User_Id', col_item='Workspace_Id', col_rating='Rating')

In [48]:
print("Model:\t",
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall,
      "RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae,
      "R2:\t%f" % eval_rsquared,
      "Exp var:\t%f" % eval_exp_var,
      "Logloss:\t%f" % eval_logloss,
      sep='\n')

Model:	
Top K:	10
MAP:	0.013686
NDCG:	0.093306
Precision@K:	0.093000
Recall@K:	0.037200
RMSE:	2.863171
MAE:	2.574562
R2:	-4.483366
Exp var:	-0.049736
Logloss:	1.103325


In [49]:
# Now let's look at the results for a specific user
user_id = 1

ground_truth = test[test['User_Id']==user_id].sort_values(by='Rating', ascending=False)[:TOP_K]
prediction = model.recommend_k_items(pd.DataFrame(dict(User_Id=[user_id])), remove_seen=True) 
final = pd.merge(ground_truth, prediction, on=['User_Id', 'Workspace_Id'], how='left')

final

2023-03-01 15:54:54,068 INFO     Calculating recommendation scores
2023-03-01 15:54:54,069 INFO     Removing seen items


,User_Id,Workspace_Id,Rating,Timestamp,Event,prediction
0,1,75,4.9,1592421822,Click,NaN
1,1,88,4.7,1594852944,Click,NaN
2,1,208,4.6,1633381808,Click,NaN
3,1,124,4.4,1676897143,Click,NaN
4,1,140,4.3,1640652539,Click,NaN
5,1,137,4.2,1643281653,Click,NaN
6,1,168,3.6,1670534454,Click,NaN
7,1,203,3.5,1617333202,Click,NaN
8,1,131,3.4,1603213550,Click,NaN
9,1,9,3.4,1676925209,Click,NaN


In [50]:
# Record results with papermill for tests - ignore this cell
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)
sb.glue("train_time", train_time.interval)
sb.glue("test_time", test_time.interval)